In [2]:
import numpy as np
import pandas as pd
import os

### Download videos for training

Downloads videos as mp4 and saves the audio as mp3. mp4 may or may not contain audio due to the API being limited, though this should not matter (except for the small negative impact on file size). Currently videos are being downloaded in 720p since 1080p is inconsistent/bugged.

In [5]:
from pytube import YouTube 
  
urls = ["https://www.youtube.com/watch?v=Yt3-a9mExZg", "https://www.youtube.com/watch?v=cGt8bEcd9Ms", 'https://www.youtube.com/watch?v=w00ooES3D5Y']
urls = ['https://www.youtube.com/watch?v=w00ooES3D5Y']
pathVideo = 'Data/Video'
pathAudio = 'Data/Audio'

for i, url in enumerate(urls):
    print('yo')
    yt = YouTube(url)
    yt.streams.filter(abr="160kbps", progressive=False).first().download(filename="Audio"+str(i)+".mp3", output_path = pathAudio)
    yt.streams.filter(res="720p", progressive=False).first().download(filename="Video"+str(i)+".mp4", output_path= pathVideo)
##

yo


AttributeError: 'NoneType' object has no attribute 'span'

### Split video into frames

Goes through each downloaded video. Creates a directory per video inside 'tempFrames' and then saves the frames here. We can set the amount of frames we want per minute. Note that file sizes of these images can balloon quickly, exceeding the filesize of the video itself.

This code is mainly for early testing

In [10]:
import cv2
FRAMES_PER_MINUTE = 10
frame_counter = 60000 / FRAMES_PER_MINUTE
pathVideo = 'Data/Video/'
videoNames = []
for filename in os.listdir(pathVideo):
    f = os.path.join(pathVideo, filename)
    if os.path.isfile(f):
        if filename != '.gitignore':
            videoNames.append(filename)

for video in videoNames:
    path = pathVideo + video
    outputPath = 'Data/tempFrames/' + video[:-4] + '/' ## make directory per video
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
        print(f'Created new dir {outputPath}')
    vidcap = cv2.VideoCapture(pathVideo + video)
    success,image = vidcap.read()
    print(f'Succesful videocapture?: {success}')
    count = 0
    while success:
        cv2.imwrite(outputPath + "%d_seconds.jpg" % (count*(frame_counter/1000)), image)     # save frame as JPEG file   
        count += 1
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_counter))   
        success,image = vidcap.read()
        

Succesful videocapture?: True
Succesful videocapture?: True


### Py-Feat

This library provides a pipeline containing multiple models related to facial detection. The most interesting aspect here is the output of AU's (action units), which will be a key part of predicting interest levels.

While the library works okay, it has difficulties with cuda and the documentation is lacking. The goal here is to use feat-py for prototyping and then later create a similar pipeline myself. This gives me more possibilities in for example fine-tuning, while also giving more control with regards to memory management and gpu usage. 

Action units cheat-sheet: https://py-feat.org/pages/au_reference.html

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import feat
from feat.facepose_detectors.img2pose.img2pose_test import Img2Pose
import torch
from torchvision import transforms
from PIL import Image
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Preprocessing.videoCropper import videoCropper
convert_tensor = transforms.ToTensor()

In [12]:
from feat.utils.io import get_test_data_path
from feat.plotting import imshow

# Helper to point to the test data folder
test_data_dir = get_test_data_path()

# Get the full path
single_face_img_path = os.path.join(test_data_dir, "single_face.jpg")

#single_face_prediction = detector.detect_image(single_face_img_path)
single_face_prediction = detector.detect_image('Data/testImages/ac3.jpg')

# Show results
single_face_prediction.plot_detections(au_barplot=False, emotion_barplot=False)
single_face_prediction

NameError: name 'detector' is not defined

In [1]:
from feat.utils.io import get_test_data_path
import os

test_data_dir = get_test_data_path()
test_video_path = os.path.join(test_data_dir, "WolfgangLanger_Pexels.mp4")
# test_video_path = "Data/Video/Video1.mp4"

# Show video
# from IPython.core.display import Video
# Video(test_video_path, embed=False)

##### Face detector and pose detector
The class `face_detector` contains a pretrained version of the `img2pose` model, which is finetuned to on frontal faces (like phone selfie cameras, webcams). It is imported through `py-feat`. It contains a `detection threshold` hyperparameter in the range of `[0,1]`. The model returns a dataframe where each row is a bounding box coordinates accompanied by 3 facial poses. We can also keep track of which frame the bounding box belongs in.

#### Video cropper
The video cropper checks the coordinates of each bounding box to determine the bounding box values closest to the top, bottom, left and right of the screen. It then crops crops the video to a lower resolution while maintaining all the neccessary information to extract features.

##### Face-tracker
The class `face_persistence_model` is based on the `face_recognition` library, using the `face_locations`, `face_encoding` and `face_landmarks `functions. `face_locations` searches a frame for faces and returns bounding box coordinates. `face_encoding` checks these faces to see if they can recognize the person. When it encounters a "new" face it saves some metadata. Then when detecting future faces it will check this new face against all the metadata files. `face_landmarks` returns 68 `x` and 68 `y` coordinates, representing the landmarks on a face. 

The inputs for the model are:
1. `TOLERANCE`: A number [0,1] that alters how the model recognizes faces. A too low tolerance leads to the same face getting detected as different unique faces, too high tolerance leads to multiple faces being detected as the same person 
2. `filename`: The name of the videofile being analyzed. Must contain extension such as ".mp4".
3. `frameSkips`: This integer value represents how often a frame is skipped, i.e. if `frameSkips=3` every 3rd frame is skipped.
4. `model`: Face location model choice of `['cnn', 'hog']`. While 'hog' is quicker (especially on CPU), 'cnn' is more accurate and utilizes GPU.
5. `modelSize`: Size of the face encoding model, choice of `['small', 'large']`.

#### AU and emotion detector
This class containg 2 models, namely a pretrained `XGB` model for detecting AU's and a `svm`/`resmasknet` model for emotion detection, which can be given as an input (`svm` is default).

The class uses facial landmarks and video footage as an input.




In [1]:
import gc
import pandas as pd
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Models.face_detection.face_detection import face_detector
from Models.emotion_detection.aus_and_emo_detection import AuEmoDetectors
from Preprocessing.videoCropper import videoCropper

filename = 'Video2_Trim_Trim.mp4'

face_detector_model = face_detector(filename=filename, detection_threshold=0.25)
output = face_detector_model.runModel()
del face_detector_model
gc.collect()

videoCrop = videoCropper(output, filename, displayVideo=False, saveCSV=True)
output = videoCrop.runModel(csvName='test')
filename = 'cropped_'+filename
del videoCrop
gc.collect()

face_tracker_model = face_persistence_model(TOLERANCE=0.6, filename=filename, model='cnn', modelSize='large')
output = face_tracker_model.runDetectionTrackOnly(output, False)
del face_tracker_model
gc.collect()



c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


Video loaded? :  True


KeyboardInterrupt: 

In [1]:
import gc
import pandas as pd
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Models.face_detection.face_detection import face_detector
from Models.emotion_detection.aus_and_emo_detection import AuEmoDetectors
from Preprocessing.videoCropper import videoCropper

filename = 'Video2_Trim_Trim.mp4'

face_detector_model = face_detector(filename=filename, detection_threshold=0.25)
output = face_detector_model.runModel()
del face_detector_model
gc.collect()

videoCrop = videoCropper(output, filename, displayVideo=False, saveCSV=True)
output = videoCrop.runModel(csvName='test')
filename = 'cropped_'+filename
del videoCrop
gc.collect()

face_tracker_model = face_persistence_model(TOLERANCE=0.6, filename=filename, model='cnn', modelSize='large')
output = face_tracker_model.runDetectionTrackOnly(output, False)
face_tracker_model.saveCSV(output)
del face_tracker_model
gc.collect()

au_emo_detector = AuEmoDetectors(filename=filename, df=output, emoModel='resmasknet', device='cpu')
output2 = au_emo_detector.runModel()

c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


Video loaded? :  True
Video loaded? :  True


c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change

In [ ]:
import numpy as np
import pandas as pd
import gc
from Models.tracking_persistence.face_tracking_persistence import face_persistence_model
from Models.face_detection.face_detection import face_detector
from Models.emotion_detection.aus_and_emo_detection import AuEmoDetectors
filename = 'Video2_Trim_Trim.mp4'
face_detector_model = face_detector(filename=filename, detection_threshold=0.25,)
output = face_detector_model.runModel()
del face_detector_model
gc.collect()
face_tracker_model = face_persistence_model(TOLERANCE=0.6, filename=filename, model='cnn', modelSize='large')
output = face_tracker_model.runDetectionTrackOnly(output, False)
face_tracker_model.saveCSV(output)
del face_tracker_model
gc.collect()
au_emo_detector = AuEmoDetectors(filename=filename, df=output, emoModel='svm')
output2 = au_emo_detector.runModel()

In [1]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'The current CUDA version is {torch.version.cuda}.')
    print(f'The GPU model is {torch.cuda.get_device_name(0)}.')
else:
    print('CUDA is not available. Running on CPU.')
print(torch.version.cuda)


c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\Ramon\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


There are 1 GPU(s) available.
The current CUDA version is 11.7.
The GPU model is NVIDIA GeForce GTX 1660 Ti.
11.7


### Video segmentation

To reduce memory usage it could be worthwile to only work with the video data inside of bounding boxes. I experimented with this in face_tracking_persistency.py

In [8]:
initialDF[1307:1313]

,Frame,Person,FaceRectX,FaceRectY,FaceRectWidth,FaceRectHeight
1307,438,0,888,40,94,95
1308,438,2,271,138,113,114
1309,438,1,547,472,114,114
1310,439,0,888,40,94,95
1311,439,2,259,138,114,114
1312,439,1,547,472,114,114


In [6]:
import numpy as np
import cv2
from feat import Detector
#emoModel = 'svm'
emoModel = 'resmasknet'
mydet = Detector(emotion_model=emoModel)

x_columns = [f'x{i}' for i in range(68)]
y_columns = [f'y{i}' for i in range(68)]
face_columns = ['FaceRectWidth', 'FaceRectHeight']
bbox_columns = ['FaceRectX', 'FaceRectWidth', 'FaceRectY', 'FaceRectHeight']
au_keys =      ["AU1","AU2","AU4","AU5","AU6","AU7","AU9","AU10","AU11","AU12","AU14",
                "AU15","AU17","AU20","AU23","AU24","AU25","AU26","AU28","AU43"]
emotion_keys = ['anger', 'disgust', 'fear',	'happiness', 'sadness',	'surprise',	'neutral']
auDF = pd.DataFrame(index=range(len(output)), columns=au_keys)
emoDF = pd.DataFrame(index=range(len(output)), columns=emotion_keys)
video = cv2.VideoCapture('./Data/Video/'+filename)
frame_i = 0
total_iterator = 0

output = pd.read_csv('facial_persistence_test_cropped_Video2_Trim_Trim.mp4_cropped.csv')


while True:
    ret, image = video.read()
    if ret == False:
        break
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    grouped = output.groupby('Frame')
    for frame, group in grouped:
        if frame == frame_i:
            frame_group = group
            break
    tempRowList = []
    for _, row in frame_group.iterrows():
        x_array = row[x_columns].tolist()
        y_array = row[y_columns].tolist()
        coordinates = list(zip(x_array, y_array))
        coordinates = np.array(coordinates, dtype=np.float32)
        nested_coordinates = [[coordinates]]
        aus = mydet.detect_aus(image, nested_coordinates)
        print(nested_coordinates)
        if emoModel == 'svm':
            emotions = mydet.detect_emotions(frame=image, facebox=['a','b'], landmarks=nested_coordinates)
        else:
            faceInfo = row[bbox_columns].tolist()
            faceInfo.append(1)
            faceInfo = [int(x) for x in faceInfo]
            faceInfo = [[faceInfo]]
            emotions = mydet.detect_emotions(frame=image, facebox=faceInfo, landmarks=nested_coordinates)
        for name, key in zip(au_keys,aus[0][0]):
            auDF.loc[total_iterator, name] = key
        for name, key in zip(emotion_keys, emotions[0][0]):
            emoDF.loc[total_iterator, name] = key
        total_iterator += 1
    
output2 = output.copy(deep=True)
output2 = pd.concat([output2, auDF, emoDF], axis=1)
output2.to_csv('final.csv', index=False)

[[array([[398., 499.],
       [399., 514.],
       [401., 529.],
       [404., 543.],
       [409., 556.],
       [418., 567.],
       [429., 573.],
       [441., 576.],
       [454., 575.],
       [466., 573.],
       [477., 568.],
       [487., 560.],
       [495., 550.],
       [500., 537.],
       [502., 522.],
       [503., 507.],
       [504., 491.],
       [405., 477.],
       [411., 469.],
       [420., 465.],
       [431., 464.],
       [440., 468.],
       [455., 468.],
       [465., 464.],
       [475., 464.],
       [484., 466.],
       [491., 473.],
       [449., 485.],
       [450., 495.],
       [451., 505.],
       [452., 516.],
       [441., 525.],
       [446., 527.],
       [452., 528.],
       [457., 526.],
       [462., 524.],
       [417., 492.],
       [422., 489.],
       [429., 488.],
       [435., 491.],
       [430., 494.],
       [423., 495.],
       [463., 491.],
       [469., 487.],
       [476., 488.],
       [481., 491.],
       [476., 493.],
       [469

KeyboardInterrupt: 